In [176]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_auc_score


In [2]:
train = pd.read_csv('../data/train.csv')
students = pd.read_csv('../data/Student.csv')
internship = pd.read_csv('../data/Internship.csv')
test = pd.read_csv('../data/test.csv')

C:\Users\Abhishek\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [183]:
X = train[['Internship_ID', 'Is_Part_Time']]
X.loc[:, 'is_hefty_compensation'] = train.Expected_Stipend.map(lambda x: int(x == '10K+'))
X.loc[:, 'is_6_month_programme'] = train.Minimum_Duration.map(lambda x: int( (x >= 5) and (x <= 6)))

y = train.Is_Shortlisted

In [184]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44) 

In [185]:
clf = GradientBoostingClassifier()
pipeline = Pipeline([('clf', clf)])

In [186]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('clf', GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])

In [187]:
predsTrain = pipeline.predict_proba(X_train)[:, 1]
predsTest = pipeline.predict_proba(X_test)[:, 1]

In [188]:
print 'AUC score on training set %f ' %(roc_auc_score(y_train, predsTrain))
print 'AUC score on test set %f ' %(roc_auc_score(y_test, predsTest))

AUC score on training set 0.691331 
AUC score on test set 0.680581 


In [189]:
pipeline.fit(X, y)

Pipeline(steps=[('clf', GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])

In [190]:
test_subset = test[['Internship_ID', 'Is_Part_Time']]
test_subset.loc[:, 'is_hefty_compensation'] = test.Expected_Stipend.map(lambda x: int(x == '10K+'))
test_subset.loc[:, 'is_6_month_programme'] = test.Minimum_Duration.map(lambda x: int( (x >= 5) and (x <= 6)))


In [191]:
predictions = pipeline.predict_proba(test_subset)[:, 1]

In [192]:
submission = pd.read_csv('../data/submission.csv')
submission['Internship_ID'] = test.Internship_ID
submission['Student_ID'] = test.Student_ID
submission['Is_Shortlisted'] = predictions

In [193]:
submission.to_csv('../submissions/gbm.csv', index=False)